In [ ]:
'''
Fine-Tuning Steps
1. Create Dataset/Dataloader (train, test, eval)
2. Instantiate pre-trained model with desired configuration
3. Specify which layers to allow for fine-tuning (opt: include adapter modules)
4. Write code for conditional inputs (see GPT_conditional_fine_tuning for motivation)
5. Write training loop (expert only)
6. Write evaluation loop
7. Visualize/check results
'''

'''
unkowns: how to handle attention masks
'''

In [ ]:

GPT_conditional_fine_tuning = "https://towardsdatascience.com/conditional-text-generation-by-fine-tuning-gpt-2-11c1a9fc639d"
gpt_fine_tune  = "https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=Z474sSC6oe7A"
gpt_fine_tune_2 = "https://medium.com/swlh/fine-tuning-gpt-2-for-magic-the-gathering-flavour-text-generation-3bafd0f9bb93"
gpt_fine_tune_3 = "https://colab.research.google.com/github/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb"


: 

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
# load gpt tokentizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # TODO: we are using different tokenizer that Jordan made? right?
# also have ability to specify some of special tokents
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

'''
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
'''
model = GPT2Model.from_pretrained('gpt2')
# can do other things like specify hyper parameters, lr scheduer
text = "Replace with topic embedding and text."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

: 

In [ ]:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i >= 6:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

# TODO: potentially add adapter modules as alternative to allowing fine-tuning on last n transformer blocks

In [4]:
type(output)

transformers.modeling_outputs.BaseModelOutputWithPastAndCrossAttentions

In [7]:
output.keys()

odict_keys(['last_hidden_state', 'past_key_values'])

In [ ]:
output = model(**encoded_input)

In [11]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_medium")
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


FileNotFoundError: Couldn't find file locally at /Users/alex3/Documents/UT/NLG/TCL-Experts-for-Control/src/models/yelp_review_medium/yelp_review_medium.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.11.0/datasets/yelp_review_medium/yelp_review_medium.py.
The file is also not present on the master branch on github.